In [3]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from augment_train import augment_train
import time
from sklearn.metrics import classification_report

urothelial_cells=pd.read_pickle("urothelial_cell_toy_data.pkl")
images=np.transpose(urothelial_cells["X"].numpy()*255,(0,2,3,1)).astype(np.uint8)
labels=urothelial_cells["y"]
X = urothelial_cells["X"].numpy()  # (200, 3, 256, 256)
Y = urothelial_cells["y"]
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train = torch.FloatTensor(X_train)  # (N, C, H, W)
X_val = torch.FloatTensor(X_val)
Y_train = torch.LongTensor(Y_train)  # (N, 1, H, W)
Y_val = torch.LongTensor(Y_val)

# Print shapes for verification
print(f"X_train shape: {X_train.shape}")  # Expected: (160, 3, 256, 256)
print(f"Y_train shape: {Y_train.shape}")  # Expected: (160, 1, 256, 256)
print(f"X_val shape: {X_val.shape}") 


X_train_aug, Y_train_aug = augment_train(X_train, Y_train, augment_times=2)

X_train shape: torch.Size([160, 3, 256, 256])
Y_train shape: torch.Size([160, 256, 256])
X_val shape: torch.Size([40, 3, 256, 256])


c:\Python312\Lib\site-packages\torch\nn\functional.py:4969: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
c:\Python312\Lib\site-packages\torch\nn\functional.py:4902: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


Augmented X shape: torch.Size([480, 3, 256, 256]), Augmented Y shape: torch.Size([480, 256, 256])


In [ ]:
#Evaluating UNet model
from semseg_functions import make_predictions
from sklearn.metrics import classification_report

begin_time=time.time()
y_val_pred=make_predictions(X_val,model=None)
y_val_pred_lbls=y_val_pred.argmax(1)
end_time=time.time()
total_time=end_time-begin_time
print(classification_report(Y_val.numpy().flatten(),y_val_pred_lbls.flatten()))
print(f"Time taken for predictions: {total_time:.2f} seconds")


In [5]:
#Evaluating TransUNet model
from semseg_functions import make_predictions_transunet



print("X_train_aug shape:", X_train_aug.shape)
print("Y_train_aug shape:", Y_train_aug.shape)

#keep track of total time
begin_time=time.time()
y_val_pred=make_predictions_transunet(X_val,model=None)
y_val_pred_lbls=y_val_pred.argmax(1)
end_time=time.time()
total_time=end_time-begin_time
print(classification_report(Y_val.numpy().flatten(),y_val_pred_lbls.flatten()))
print(f"Time taken for predictions: {total_time:.2f} seconds")


from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize

# Flatten prediction: shape (B, C, H, W) → (N, C)
y_val_pred_flat = y_val_pred.transpose(0, 2, 3, 1).reshape(-1, y_val_pred.shape[1])

# Flatten true labels: shape (B, H, W) → (N,)
y_true = Y_val.numpy().flatten()

# Binarize true labels for multiclass AUROC
y_true_binarized = label_binarize(y_true, classes=[0, 1, 2])

# Compute AUROC
auroc = roc_auc_score(y_true_binarized, y_val_pred_flat, multi_class='ovr')

print("AUROC:", auroc)



X_train_aug shape: torch.Size([480, 3, 256, 256])
Y_train_aug shape: torch.Size([480, 256, 256])


c:\Users\Chris\Documents\MasterProject590\code\demos\Project2\semseg_functions.py:315: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_l

Loading model from: ./seg_models\best_transunet_model.pkl
              precision    recall  f1-score   support

           0       0.98      0.95      0.96    937035
           1       0.92      0.86      0.89   1187442
           2       0.75      0.92      0.83    496963

    accuracy                           0.90   2621440
   macro avg       0.88      0.91      0.89   2621440
weighted avg       0.91      0.90      0.90   2621440

Time taken for predictions: 1.55 seconds
AUROC: 0.9827075787794297


In [ ]:
#Evaluating Intensity thresholding